# Examing Data from COVID-19 Outbreak

#### Import Libraries

In [ ]:
import pandas as pd
from datetime import date, timedelta
import xlrd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

pd.options.mode.chained_assignment = None  # default='warn'

#### Import Latest XLS from ECDC website

In [ ]:
today = date.today()

In [ ]:
day = today.strftime("%d")
month = today.strftime("%m")

In [ ]:
url  = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-{}-{}.xlsx".format(month, day)

In [ ]:
url

#### Load into DataFrame

In [ ]:
df = pd.read_excel(url)

In [ ]:
df

In [ ]:
#df.info()

#### Output DataFrame to CSV

In [ ]:
#Saves local coopy of ECDC 

date = today.strftime('%Y-%m-%d')

filename = "ECDC Data " + date + '.csv'
filename

In [ ]:
df.to_csv(filename)

#### Fix Date format

In [ ]:
#Excel to DateTime
#def read_date(date):
#return xlrd.xldate.xldate_as_datetime(date, 0)

In [ ]:
#df['dateRep'] = pd.to_datetime(df['dateRep'].apply(read_date), errors='coerce')

In [ ]:
#Obj to DateTime
#df['dateRep'] = pd.to_datetime(df[['year', 'month', 'day']])

In [ ]:
#df

In [ ]:
#df.info()

#### Set Date as Index

In [ ]:
df.set_index('dateRep', inplace=True)
df = df.sort_index()

In [ ]:
df

In [ ]:
countries = df['countriesAndTerritories'].values
np.unique(countries)

#### Fix Inconsistent Capitalisation of Country Names

In [ ]:
#df.replace(['United kingdom', 'Czech republic', 'switzerland', 'CANADA'], ['United Kingdom', 'Czech Republic', 'Switzerland', 'Canada'], inplace=True)

In [ ]:
#countries = df['countriesAndTerritories'].values
#np.unique(countries)

#### Set up Seven Day Window

In [ ]:
seven_day = today - timedelta(days = 7)

#### Examine Global Numbers

In [ ]:
df_global = df[['cases', 'deaths']].groupby(df.index).sum()

In [ ]:
df_global.head()

In [ ]:
df_global['Total_Cases'] = df_global['cases'].cumsum()
df_global['Total_Deaths'] = df_global['deaths'].cumsum()
df_global['SDA_Cases'] = df_global['cases'].rolling(window=7).mean()
df_global['SDA_Deaths'] = df_global['deaths'].rolling(window=7).mean()

In [ ]:
df_global

#### Total Confirmed Cases and Deaths Globally

In [ ]:
df_global.cases.sum()

In [ ]:
df_global.deaths.sum()

In [ ]:
#plot data
fig, ax = plt.subplots(figsize=(17,7))
ax.plot(df_global.index, df_global['Total_Cases'])
ax.plot(df_global.index, df_global['Total_Deaths'])

#set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

# Turn on the grid
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5', color='black')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='red')
ax.set_ylabel('No. of People')

plt.title("Total No. of Confirmed Cases and Deaths Globally")

plt.legend(['Total Confirmed Cases', 'Total Deaths'])

plt.savefig('Images/' + str(today) + ' Total No. of Confirmed Cases and Deaths Globally.png')

plt.show()

#### Daily New Confirmed Cases and Deaths Globally

In [ ]:
#plot data
fig, ax = plt.subplots(figsize=(17,7))
ax.bar(df_global.index, df_global['cases'])
ax.bar(df_global.index, df_global['deaths'])
ax.plot(df_global.index, df_global['SDA_Cases'],color='y', linewidth = 2)
ax.plot(df_global.index, df_global['SDA_Deaths'],color='m', linewidth = 2)

#set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

# Turn on the grid
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5', color='black')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='red')
ax.set_ylabel('No. of People')

plt.title("Daily No. of New Confirmed Cases and Deaths Globally")

plt.legend(['7-Day Mean Cases', '7-Day Mean Deaths','New Confirmed Cases', 'New Deaths'], loc=2)

plt.savefig('Images/' + str(today) + ' Daily No. of New Confirmed Cases and Deaths Globally.png')

plt.show()

#### Top 20 Countries with the Most Confirmed Cases and Deaths

In [ ]:
df_countries = df.set_index('countriesAndTerritories')

In [ ]:
df_countries = df_countries.groupby('countriesAndTerritories').sum()

In [ ]:
df_countries = df_countries.sort_values('cases', ascending=True)

In [ ]:
#plot data
labels = df_countries.index[-20:]
Cases = df_countries['cases'][-20:]
Deaths = df_countries['deaths'][-20:]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(17, 7))
rects1 = ax.bar(x - width/2, Cases, width, label='New Confirmed Cases')
rects2 = ax.bar(x + width/2, Deaths, width, label='New Deaths')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('No. of People')
ax.set_title('Top 20 Countries with the Most Confirmed Cases and Deaths')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=90)

ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='red')
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.savefig('Images/' + str(today) + ' Top 20 Countries with the Most Confirmed Cases and Deaths.png')

plt.show()

  #### Find Latest Data from Yesterday

In [ ]:
yesterday = today - timedelta(days = 1)

In [ ]:
df_yesterday = df.loc[yesterday]
df_yesterday.set_index('countriesAndTerritories', inplace=True)

In [ ]:
df_yesterday = df_yesterday.sort_values('cases', ascending=True)

In [ ]:
df_yesterday

#### Total No. of Confirmed Cases and Deaths Yesterday Globally

In [ ]:
#### Note: I don't feel that this barchart adds anything to the analysis, so I'm discontinuing it as of 24/05

In [ ]:
# df_yesterday.cases.sum()

In [ ]:
# df_yesterday.deaths.sum()

In [ ]:
# #plot data
# labels = ['Yesterday']
# Cases = df_yesterday.cases.sum()
# Deaths = df_yesterday.deaths.sum()

# x = np.arange(len(labels))  # the label locations
# # width = 0.35  # the width of the bars

# fig, ax = plt.subplots(figsize=(7, 7))
# rects1 = ax.bar(x - width/2, Cases, width, label='Total New Confirmed Cases')
# rects2 = ax.bar(x + width/2, Deaths, width, label='Total New Deaths')

# # Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_ylabel('No. of People')
# ax.set_title('Total No. of Confirmed Cases and Deaths Globally Yesterday (' + str(yesterday) + ')')
# ax.set_xticks(x)
# ax.set_xticklabels(labels)

# ax.minorticks_on()
# ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='red')
# ax.legend()

# autolabel(rects1)
# autolabel(rects2)

# fig.tight_layout()

# plt.savefig('Images/' + str(today) + ' Total No. of Confirmed Cases and Deaths Globally Yesterday (' + str(yesterday) + ').png')

# plt.show()

#### Top 20 Countries with the Most New Confirmed Cases and Deaths Yesterday

In [ ]:
#plot data
labels = df_yesterday.index[-20:]
Cases = df_yesterday['cases'][-20:]
Deaths = df_yesterday['deaths'][-20:]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(17, 7))
rects1 = ax.bar(x - width/2, Cases, width, label='New Confirmed Cases')
rects2 = ax.bar(x + width/2, Deaths, width, label='New Deaths')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('No. of People')
ax.set_title('Top 20 Countries with the Most New Confirmed Cases and Deaths Yesterday (' + str(yesterday) + ')')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=90)

ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='red')
ax.legend()

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.savefig('Images/' + str(today) + ' Top 20 Countries with the Most New Confirmed Cases and Deaths Yesterday ('
            + str(yesterday) + ').png')

plt.show()

#### Examine Cases and Deaths Across EU

In [ ]:
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech_Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia',
       'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland',
       'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']

In [ ]:
df_eu = df[df['countriesAndTerritories'].isin(eu_countries)]

In [ ]:
df_eu

In [ ]:
df_eu = df_eu.loc[:,('cases', 'deaths')]

In [ ]:
df_eu = df_eu.groupby('dateRep').sum()
df_eu['SDA_Cases'] = df_eu['cases'].rolling(window=7).mean()
df_eu['SDA_Deaths'] = df_eu['deaths'].rolling(window=7).mean()

In [ ]:
df_eu

In [ ]:
df_eu_sum = df_eu.groupby('dateRep').sum().cumsum()
df_eu_sum.drop(['SDA_Cases', 'SDA_Deaths'], axis=1, inplace=True)

df_eu_sum

In [ ]:
#plot data
fig, axs = plt.subplots(2, 1, figsize=(17, 10))
axs[0].bar(df_eu.index, df_eu['cases'])
axs[0].bar(df_eu.index, df_eu['deaths'])
axs[0].plot(df_eu.index, df_eu['SDA_Cases'],color='y', linewidth = 2)
axs[0].plot(df_eu.index, df_eu['SDA_Deaths'],color='m', linewidth = 2)

#set ticks every week
axs[0].xaxis.set_major_locator(mdates.WeekdayLocator())
axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

#Gridlines
axs[0].minorticks_on()
axs[0].grid(which='major', linestyle='-', linewidth='0.5', color='black')
axs[0].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
axs[0].set_ylabel('No. of People')


#Titles
axs[0].title.set_text("Daily No. of New Confirmed Cases and Deaths in the EU")

#Legends
axs[0].legend(['7-Day Mean Cases', '7-Day Mean Deaths', 'New Confirmed Cases', 'New Deaths'], loc=2)

#Total Cases in EU

#plot data
axs[1].plot(df_eu_sum.index, df_eu_sum['cases'])
axs[1].plot(df_eu_sum.index, df_eu_sum['deaths'])

#set ticks every week
axs[1].xaxis.set_major_locator(mdates.WeekdayLocator())
axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

#Gridlines
axs[1].minorticks_on()
axs[1].grid(which='major', linestyle='-', linewidth='0.5', color='black')
axs[1].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
axs[1].set_ylabel('No. of People')

axs[1].minorticks_on()
axs[1].grid(which='major', linestyle='-', linewidth='0.5', color='black')
axs[1].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
axs[1].set_ylabel('No. of People')


#Titles
axs[1].title.set_text("Total No. of Confirmed Cases and Deaths in the EU")

#Legends
axs[1].legend(['Total Confirmed Cases', 'Total Deaths'], loc=2)

plt.savefig('Images/'+str(today) + ' Daily & Total No. of Confirmed Cases and Deaths in the EU.png')

plt.show()

#### Find Data for Specific Countries (e.g. DE, UK, CN) and Compare

#### Function: *Daily No. of New Confirmed Cases and Deaths in X & Y*

In [ ]:
def compare_daily_cases(country1, country2):
    df_sub1 = df.loc[df['geoId'] == country1]
    df_sub2 = df.loc[df['geoId'] == country2]
    df_sub1['SDA_Cases'] = df_sub1['cases'].rolling(window=7).mean()
    df_sub1['SDA_Deaths'] = df_sub1['deaths'].rolling(window=7).mean()
    df_sub2['SDA_Cases'] = df_sub2['cases'].rolling(window=7).mean()
    df_sub2['SDA_Deaths'] = df_sub2['deaths'].rolling(window=7).mean()
    

    #Plot Data for country1
    fig, axs = plt.subplots(2, 1, figsize=(17, 10))
    axs[0].bar(df_sub1.index, df_sub1['cases'])
    axs[0].bar(df_sub1.index, df_sub1['deaths'])
    axs[0].plot(df_sub1.index, df_sub1['SDA_Cases'],color='y', linewidth = 2)
    axs[0].plot(df_sub1.index, df_sub1['SDA_Deaths'],color='m', linewidth = 2)
    

    #Plot Data for country2
    axs[1].bar(df_sub2.index, df_sub2['cases'])
    axs[1].bar(df_sub2.index, df_sub2['deaths'])
    axs[1].plot(df_sub2.index, df_sub2['SDA_Cases'],color='y', linewidth = 2)
    axs[1].plot(df_sub2.index, df_sub2['SDA_Deaths'],color='m', linewidth = 2)


    #set ticks every week
    axs[0].xaxis.set_major_locator(mdates.WeekdayLocator())
    axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    axs[1].xaxis.set_major_locator(mdates.WeekdayLocator())
    axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

    #Gridlines
    axs[0].minorticks_on()
    axs[0].grid(which='major', linestyle='-', linewidth='0.5', color='black')
    axs[0].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
    axs[0].set_ylabel('No. of People')

    axs[1].minorticks_on()
    axs[1].grid(which='major', linestyle='-', linewidth='0.5', color='black')
    axs[1].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
    axs[1].set_ylabel('No. of People')


    #Titles
    title1 = df_sub1['countriesAndTerritories'].unique()[0]
    title2 = df_sub2['countriesAndTerritories'].unique()[0]
    axs[0].title.set_text("Daily No. of New Confirmed Cases and Deaths in " + title1)
    axs[1].title.set_text("Daily No. of New Confirmed Cases and Deaths in " + title2)

    #Legends
    axs[0].legend(['7-Day Mean Cases', '7-Day Mean Deaths', 'New Confirmed Cases', 'New Deaths'], loc=2)
    axs[1].legend(['7-Day Mean Cases', '7-Day Mean Deaths', 'New Confirmed Cases', 'New Deaths'], loc=2)

    plt.savefig('Images/' + str(today) + ' Daily No. of New Confirmed Cases and Deaths in ' + title1 + ' & ' + title2 + '.png')

    plt.show()

#### Function: *Total No. of Confirmed Cases and Deaths in X & Y*

In [ ]:
def compare_total_cases(country1, country2):
    
    df_sub1 = df.loc[df['geoId'] == country1]
    df_sub2 = df.loc[df['geoId'] == country2]
    
    df_sub1['Total_Cases'] = df_sub1.loc[:,('cases')].cumsum()
    df_sub1['Total_Deaths'] = df_sub1.loc[:,('deaths')].cumsum()

    df_sub2['Total_Cases'] = df_sub2.loc[:,('cases')].cumsum()
    df_sub2['Total_Deaths'] = df_sub2.loc[:,('deaths')].cumsum()


    #plot data
    fig, axs = plt.subplots(2, 1, figsize=(17, 10))
    axs[0].plot(df_sub1.index, df_sub1['Total_Cases'])
    axs[0].plot(df_sub1.index, df_sub1['Total_Deaths'])

    #Set 
    axs[1].plot(df_sub2.index, df_sub2['Total_Cases'])
    axs[1].plot(df_sub2.index, df_sub2['Total_Deaths'])

    #set ticks every week
    axs[0].xaxis.set_major_locator(mdates.WeekdayLocator())
    axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    axs[1].xaxis.set_major_locator(mdates.WeekdayLocator())
    axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

    #Gridlines
    axs[0].minorticks_on()
    axs[0].grid(which='major', linestyle='-', linewidth='0.5', color='black')
    axs[0].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
    axs[0].set_ylabel('No. of People')

    axs[1].minorticks_on()
    axs[1].grid(which='major', linestyle='-', linewidth='0.5', color='black')
    axs[1].grid(which='minor', linestyle=':', linewidth='0.5', color='red')
    axs[1].set_ylabel('No. of People')


    #Titles
    title1 = df_sub1['countriesAndTerritories'].unique()[0]
    title2 = df_sub2['countriesAndTerritories'].unique()[0]
    axs[0].title.set_text("Total No. of Confirmed Cases and Deaths in " + title1)
    axs[1].title.set_text("Total No. of Confirmed Cases and Deaths in " + title2)

    #Legends
    axs[0].legend(['Total Confirmed Cases', 'Total Deaths'], loc=2)
    axs[1].legend(['Total Confirmed Cases', 'Total Deaths'], loc=2)

    plt.savefig('Images/'+str(today) + ' Total No. of Confirmed Cases and Deaths in ' + title1 + ' & ' + title2 + '.png')

    plt.show()

#### Compare Charts

In [ ]:
#BR-RU, CN-IR, DE-FR, IT-ES, IL-TR, JP-KR, IN-PK, US-UK, SE-NL

country1 = 'SE'
country2 = 'UK'

In [ ]:
compare_daily_cases(country1,country2)
compare_total_cases(country1,country2)